<a href="https://colab.research.google.com/github/ethansong206/Climate-Plus-Project/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary packages for the code

In [36]:
import pandas as pd
import numpy as np
from datetime import datetime

Load the Duke Dining csv file and make the data easier to handle. This includes but is not limited to:

1) Renaming locations

2) Changing the date from a string to a date object



In [67]:
DiningDataFull = pd.read_csv('Climate+ Data 2019 thru 2023.csv')
#print(DiningDataFull['Unit Name'].value_counts()) # Show how many entries there are of each location

#below code to make 'Unit Name' column easier to handle
#can add more lines given more locations
def location_rename(location):
    if("Marketplace" in location): #Combine data for Marketplace Kitchen and Marketplace Special Event
        return "Marketplace"
    if("Marine Lab" in location):
        return "DuML"
    if("Trinity" in location):
        return "Trinity"
    if("Freeman" in location):
        return "Freeman"
    return None

DiningDataFull['Unit Name'] = DiningDataFull.apply(lambda d: location_rename(d['Unit Name']), axis = 1)

DiningDataFull['Purchase Date'] = DiningDataFull.apply(lambda d: datetime.strptime(d['Purchase Date'], '%m/%d/%Y').date(), axis = 1)

print(DiningDataFull.head())

<ipython-input-67-59ebf3b3dbb7>:1: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  DiningDataFull = pd.read_csv('Climate+ Data 2019 thru 2023.csv')


     Unit Name    Vendor Purchase Date Vendor Item ID  \
0  Marketplace  US Foods    2019-07-01        5018999   
1  Marketplace  US Foods    2019-07-01        5400692   
2  Marketplace  US Foods    2019-07-01        8012643   
3  Marketplace  US Foods    2019-07-01         731349   
4  Marketplace  US Foods    2019-07-01          68106   

                             Vendor Item Description  \
0  POTATO, FRENCH-FRY 3/4 X3/8 STEAK CUT SKIN-ON ...   
1  POTATO, FRENCH-FRY SPIRAL COATED SEASONED TFF ...   
2  POTATO, FRENCH-FRY 1/2 CRINKLE-CUT TFF EXTRA-L...   
3                POTATO, HASH BROWN SHRED COOKED REF   
4    POTATO, TATER NUGGET PARFRIED FROZEN HASH BROWN   

  Vendor Item Purchase Unit                              Product Group  \
0                    6/5 LB  APPETIZERS, ENTREES, & POTATOES REF & FZN   
1                    6/5 LB  APPETIZERS, ENTREES, & POTATOES REF & FZN   
2                  6/4.5 LB  APPETIZERS, ENTREES, & POTATOES REF & FZN   
3                   2/10

Making one dataset for each location and each year

In [76]:
location_col = DiningDataFull['Unit Name']
DiningData_Marketplace = DiningDataFull[location_col == "Marketplace"]
DiningData_DuML = DiningDataFull[location_col == "DuML"]
DiningData_Trinity = DiningDataFull[location_col == "Trinity"]
DiningData_Freeman = DiningDataFull[location_col == "Freeman"]

date_col = DiningDataFull['Purchase Date']
DiningData_2019 = DiningDataFull[(date_col >= pd.Timestamp(2019, 1, 1)) & (date_col < pd.Timestamp(2020, 1, 1))]
DiningData_2020 = DiningDataFull[(date_col >= pd.Timestamp(2020, 1, 1)) & (date_col < pd.Timestamp(2021, 1, 1))]
DiningData_2021 = DiningDataFull[(date_col >= pd.Timestamp(2021, 1, 1)) & (date_col < pd.Timestamp(2022, 1, 1))]
DiningData_2022 = DiningDataFull[(date_col >= pd.Timestamp(2022, 1, 1)) & (date_col < pd.Timestamp(2023, 1, 1))]
DiningData_2023 = DiningDataFull[(date_col >= pd.Timestamp(2023, 1, 1)) & (date_col < pd.Timestamp(2024, 1, 1))]

<ipython-input-76-7747d46d14ce>:8: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  DiningData_2019 = DiningDataFull[(date_col >= pd.Timestamp(2019, 1, 1)) & (date_col < pd.Timestamp(2020, 1, 1))]
<ipython-input-76-7747d46d14ce>:9: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  DiningData_2020 = DiningDataFull[(date_col >= pd.Timestamp(2020, 1, 1)) & (date_col < pd.Timestamp(2021, 1, 1))]
<ipython-input-76-7747d46d14ce>:10: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comp

         Unit Name    Vendor Purchase Date Vendor Item ID  \
77182  Marketplace  US Foods    2023-01-04        8959935   
77183  Marketplace  US Foods    2023-01-06        4818803   
77184  Marketplace  US Foods    2023-01-06        2341238   
77185  Marketplace  US Foods    2023-01-06        4985693   
77186  Marketplace  US Foods    2023-01-06        3587490   
...            ...       ...           ...            ...   
90529         DuML  US Foods    2023-05-23        8744005   
90530         DuML  US Foods    2023-05-23        6000152   
90531         DuML  US Foods    2023-05-23        1301084   
90532         DuML  US Foods    2023-05-23        8879942   
90533         DuML  US Foods    2023-05-24        5281756   

                                 Vendor Item Description  \
77182  BEEF, TENDERLOIN CANADIAN PRIME 189A PEELED-SI...   
77183  WRAP, 12X12 WAX PAPER NATURAL FLAT PACK GREASE...   
77184  CREAMER, HALF & HALF 10.5% BUTTERFAT UHT DAIRY...   
77185  CHEESE, CHEDDAR MILD

Make a new column called `Unit(g)` for converting all units to the equivalent value in grams, then filter out main descriptor word(s) in `Vendor Item Description` into a new column called `Food Name`.

Take the column with units in grams and multiply by the `Receive Quantity` if provided into a new column called `Total Grams`
